In [4]:
#import needed package
import asyncio
import httpx
import pandas as pd
import secret
import refresh
import requests
from time import perf_counter
from datetime import datetime

In [5]:
#credentials
id_token = refresh.refreshed()
headers = {"content-type":"application/json", "x-api-key": secret.api_key, "id-token": id_token}

In [6]:
#read excel file
df = pd.read_excel('hubspotid_update_140223.xlsx')
df.head()

,Record ID,Vincere ID
0,17530871,73520
1,17530821,82637
2,17530721,82587
3,17530671,82586
4,17530622,82585


In [10]:
#sem_GET = asyncio.Semaphore(100)
sem_PUT = asyncio.Semaphore(10)

#changing "key" field to "field_key"
def change_dict(data: dict) -> dict:
    if data:
        data_update = {'data':[{}]}
        for i in data:
            i['field_key'] = i.pop('key')
        data_update['data'] = data
        return data_update
    else:
        return {}

#removing the "field key" data that is going to be updated
def data_added(data: dict, hs_id: int) -> dict:
    if data:
        data_container = [d for d in data if d['key'] != "6c04623e10d39f08d1b2542e138b7efc"]
        data_remove = [d for d in data if d['key'] == "6c04623e10d39f08d1b2542e138b7efc"]
        data_remove[0]['field_value'] = hs_id
        data_container.append(data_remove[0])
        return data_container
    else:
        return {}
    
#determining if it has the custom field data
def add_custom(data: dict) -> dict:
    data_container = [d for d in data if d['key'] == "6c04623e10d39f08d1b2542e138b7efc"]
    if data_container == []:
        data_cont = {'key': '6c04623e10d39f08d1b2542e138b7efc','type': 'TEXT_BOX','field_value_ids': None,'field_values': None,
                'field_value': 0,'date_value': None,'element_value_groups': None}
        data.append(data_cont)
        return data
    else:
        return data 
        
#requesting data first
def request_data(id: int) -> dict:
    with httpx.Client() as client:
        r = client.get(f'https://bps-world.vincere.io/api/v2/candidate/{id}/customfields', headers=headers)
        if r.status_code == 200:
            response = add_custom(r.json())
            return response
        else:
            errorID_get[id] = r.status_code
            return {}

#update data
async def update_data(id: int, hs_id: int) -> None:
    async with sem_PUT:
        async with httpx.AsyncClient() as client:
            response_data = request_data(id)
            updated_data = data_added(response_data, hs_id)
            data_container = change_dict(updated_data)
            r = await client.put(f'https://bps-world.vincere.io/api/v2/candidate/{id}/customfields', headers=headers, json=data_container)
            if r.status_code == 200:
                success[id] = r.status_code
                return
            else:
                errorID_put[id] = r.status_code
                return
                
#store errors for future reference
errorID_get = {}
errorID_put = {}
success = {}

async def main():
    vincere_id = 85738
    record_id = 333333
    await update_data(vincere_id, record_id)
#     for x, y in zip(df['Vincere ID'], df['Record ID']):
#         await update_data(x, y)

In [11]:
currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%H:%M:%S")
print(currentTime)

20:54:35


In [12]:
#trigger the whole function
start = perf_counter()
await main()
stop = perf_counter()
print('time taken:', stop - start, ' seconds')

time taken: 2.3784418000000187  seconds


In [46]:
len(success)

12

In [47]:
success

{78948: 200,
 78836: 200,
 78703: 200,
 78701: 200,
 78659: 200,
 78581: 200,
 78500: 200,
 78484: 200,
 78458: 200,
 78354: 200,
 78350: 200,
 73704: 200}

In [48]:
list(success)[-1]

73704

In [10]:
errorID_get

{}

In [11]:
list(errorID_put)[-1]

IndexError: list index out of range

In [12]:
list(errorID_get)[-1]

IndexError: list index out of range

In [39]:
r = requests.get('https://bps-world.vincere.io/api/v2/candidate/81650/customfields', headers=headers)
r.status_code

200

In [40]:
r.json()

[{'key': '6c04623e10d39f08d1b2542e138b7efc',
  'type': 'TEXT_BOX',
  'field_value_ids': None,
  'field_values': None,
  'field_value': '19288375',
  'date_value': None,
  'element_value_groups': None}]